In [1]:
from p_drought_indices.analysis.DeepLearning.pipeline_gwnet import data_preparation 
from p_drought_indices.analysis.DeepLearning.pipeline_convlstm import training_lstm
import pickle
import os
import matplotlib.pyplot as plt
from p_drought_indices.functions.function_clns import load_config, prepare, CNN_split, interpolate_prepare
import numpy as np
from p_drought_indices.analysis.DeepLearning.dataset import CustomDataset
from torch.utils.data import DataLoader

CONFIG_PATH = "../config.yaml"
sub_precp, ds = data_preparation(CONFIG_PATH)
sub_precp = sub_precp.to_dataset()
data, target = interpolate_prepare(sub_precp, ds)

c:\Users\Riccardo\anaconda3\envs\gis2_py39\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


The ERA5 raster has spatial dimensions: (0.10000000419197502, -0.1000000004915847)
NDVI dataset resolution: (0.043782177927351104, 0.04378217792735114)
Precipitation dataset resolution (0.10000000419197502, -0.1000000004915847)


In [3]:
train_split = 0.8
training_lstm(CONFIG_PATH, data, target, train_split = train_split)

c:\Users\Riccardo\Desktop\Other\package\Indices_analysis\notebooks\.


TypeError: cannot directly convert an xarray.Dataset into a numpy array. Instead, create an xarray.DataArray first, either with indexing on the Dataset or by invoking the `to_array()` method.

In [16]:
train_split = 0.8
batch_size = 32

train_data, test_data, train_label, test_label = CNN_split(data=data, target=target, split_percentage=train_split)

# create a CustomDataset object using the reshaped input data
train_dataset = CustomDataset(train_data, train_label)
test_dataset = CustomDataset(test_data, test_label)

# create a DataLoader object that uses the dataset
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

### check shape of data
for batch_idx, (inputs, targets) in enumerate(train_dataloader):
    inputs = inputs.float()
    targets = targets.float()
    print(inputs.shape, targets.shape, inputs.max(), inputs.min())


for batch_idx, (inputs, targets) in enumerate(test_dataloader):
    inputs = inputs.float()
    targets = targets.float()
    print(inputs.shape, targets.shape, inputs.max(), inputs.min())

The input data has shape: (1, 730, 1, 64, 64)
The input data has shape: (1, 730, 1, 64, 64)
torch.Size([1, 584, 1, 64, 64]) torch.Size([1, 584, 1, 64, 64]) tensor(309.7275) tensor(0.)
torch.Size([1, 146, 1, 64, 64]) torch.Size([1, 146, 1, 64, 64]) tensor(252.2191) tensor(0.)


In [4]:
#### Start training
from p_drought_indices.configs.config_3x3_16_3x3_32_3x3_64 import config
from torch.nn import MSELoss
import matplotlib.pyplot as plt
from p_drought_indices.analysis.DeepLearning.ConvLSTM import ConvLSTM, train_loop, valid_loop, build_logging
import numpy as np
import torch

name = '3x3_16_3x3_32_3x3_64'

logger = build_logging(config)
model = ConvLSTM(config).to(config.device)
#criterion = CrossEntropyLoss().to(config.device)
#criterion = torch.nn.MSELoss().to(config.device)
criterion = MSELoss().to(config.device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train_records, valid_records, test_records = [], [], []
for epoch in range(config.epochs):
    epoch_records = train_loop(config, logger, epoch, model, train_dataloader, criterion, optimizer)
    train_records.append(np.mean(epoch_records['loss']))
    epoch_records = valid_loop(config, logger, epoch, model, test_dataloader, criterion)
    valid_records.append(np.mean(epoch_records['loss']))
    plt.plot(range(epoch + 1), train_records, label='train')
    plt.plot(range(epoch + 1), valid_records, label='valid')
    plt.legend()
    plt.savefig(os.path.join(config.output_dir, '{}.png'.format(name)))
    plt.close()


c:\Users\Riccardo\Desktop\Other\package\Indices_analysis\notebooks\.
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
tensor(309.7275)
